In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import nbformat
import plotly.io as pio

In [ ]:
# Lecture du dataset
df=pd.read_csv('ratios_inpi_bce.csv', sep=';')
pd.options.display.max_columns = None
df.head(5)

In [ ]:
df.columns.tolist()

---

#### Statistiques

In [ ]:
print("Number of rows : {}".format(df.shape[0]))
print()

print("Basics statistics: ")
df_desc = df.describe(include="all")
display(df_desc)

In [ ]:
df.dtypes

In [ ]:
# Valeurs nulles
(df.isnull().sum() / len(df)) * 100

In [ ]:
df["confidentiality"].unique()

In [ ]:
nb_ca_negatif = (df["Chiffre_d_affaires"] < 0).sum()
print(f"Nombre de lignes avec un CA négatif : {nb_ca_negatif}")

In [ ]:
df[df["Chiffre_d_affaires"] < 0].head(10)

In [ ]:
# Sélection des lignes avec CA négatif
df_ca_neg = df[df["Chiffre_d_affaires"] < 0]

# Compter le nombre de bilans négatifs par SIREN
ca_neg_par_siren = df_ca_neg["siren"].value_counts().reset_index()
ca_neg_par_siren.columns = ["siren", "nb_bilans_negatifs"]

# Trier par le plus grand nombre de bilans négatifs
ca_neg_par_siren = ca_neg_par_siren.sort_values(by="nb_bilans_negatifs", ascending=False)

ca_neg_par_siren.head(10)


In [ ]:
# Moyenne du CA négatif par SIREN
ca_neg_moyenne = df_ca_neg.groupby("siren")["Chiffre_d_affaires"].mean().reset_index()
ca_neg_moyenne.columns = ["siren", "ca_moyen_negatif"]

# Fusionner avec le nombre de bilans négatifs
ca_neg_summary = ca_neg_par_siren.merge(ca_neg_moyenne, on="siren")

# Trier par CA moyen le plus négatif
ca_neg_summary = ca_neg_summary.sort_values(by="ca_moyen_negatif")

ca_neg_summary


---

#### Vérification des doublons

In [ ]:
# Vérification des doublons
print(f"Nombre de doublons : {df.duplicated().sum()}")

In [ ]:
# Nombre de doublons uniquement sur la colonne SIREN
nombre_doublons_siren = df.duplicated(subset=["siren"]).sum()

print(f"Nombre de doublons sur les SIREN : {nombre_doublons_siren}")

---

In [ ]:
# Filtrer uniquement les bilans publics
df_public = df[df["confidentiality"] == "Public"].copy()

print(f"Nombre de lignes publiques : {len(df_public)}")

In [ ]:
df_public.shape

---

#### Tri par SIREN et CA moyens

In [ ]:
# Moyenne du Chiffre d'Affaires par SIREN
ca_moyen_par_siren = (
    df_public.groupby("siren")["Chiffre_d_affaires"]
      .mean()
      .reset_index()
)

In [ ]:
ca_moyen_par_siren_sorted = ca_moyen_par_siren.sort_values(
    by="Chiffre_d_affaires",
    ascending=False
).reset_index(drop=True)

In [ ]:
ca_moyen_par_siren_sorted.head(10)

In [ ]:
ca_moyen_par_siren_sorted.shape

---

In [ ]:
ca_moyen_par_siren_sorted.describe()

---

---

In [ ]:
ca = df['Chiffre_d_affaires']
ca_clean = ca[ca > 0]

In [ ]:
# Définir des tranches logarithmiques
bins = [0, 1e3, 1e5, 1e6, 1e7, 1e8, 1e9, 1e10, 1e12, ca_clean.max()]

labels = [
    "<1k", "1k-100k", "100k-1M", "1M-10M", "10M-100M",
    "100M-1B", "1B-10B", "10B-1T", ">1T"
]

# Créer la colonne tranche
df['CA_tranche'] = pd.cut(df['Chiffre_d_affaires'], bins=bins, labels=labels, include_lowest=True)

# Comptage par tranche
ca_distribution = df['CA_tranche'].value_counts().sort_index()
ca_distribution_percent = (ca_distribution / len(df)) * 100

# Tableau final
distribution_table = pd.DataFrame({
    'Nombre d\'entreprises': ca_distribution,
    'Pourcentage (%)': ca_distribution_percent.round(2)
})

print(distribution_table)

In [ ]:
ca_distribution = df['CA_tranche'].value_counts().sort_index()
pie_df = ca_distribution.reset_index()
pie_df.columns = ['Tranche', 'Nombre d\'entreprises']

# Pie chart interactif
fig = px.pie(
    pie_df,
    names='Tranche',
    values='Nombre d\'entreprises',
    title="Répartition des entreprises par tranches de Chiffre d'Affaires",
    hole=0
)

# Légende à côté
fig.update_layout(
    width=900,
    height=700,
    legend=dict(
        orientation="v",
        yanchor="top",
        y=1,
        xanchor="left",
        x=1.05
    )
)

fig.show()

In [ ]:
fig = px.bar(
    pie_df,
    x='Tranche',
    y="Nombre d'entreprises",
    text="Nombre d'entreprises",
    title="Répartition des entreprises par tranches de Chiffre d'Affaires"
)

fig.update_traces(texttemplate='%{text:,}', textposition='outside')
fig.update_layout(
    width=1000,
    height=600,
    yaxis_title="Nombre d'entreprises",
    xaxis_title="Tranche de CA"
)

fig.show()

---

#### Test de tri pour les sociétés entre 1K et 10M

In [ ]:
def affiner_tranche(row):
    ca = row["Chiffre_d_affaires"]
    tranche = row["CA_tranche"]

    if tranche == "1k-100k":
        if ca <= 15_000:
            return "1k-15k"
        else:
            return "15k-100k"
    else:
        return tranche


In [ ]:
df["CA_tranche_affinee"] = df.apply(affiner_tranche, axis=1)


In [ ]:
tranches_cible = [
    "1k-15k",
    "15k-100k",
    "100k-1M",
    "1M-10M"
]

df_filtre = df[df["CA_tranche_affinee"].isin(tranches_cible)]


In [ ]:
ca_distribution = (
    df_filtre["CA_tranche_affinee"]
    .value_counts()
    .reindex(tranches_cible)
)


In [ ]:
distribution_table = pd.DataFrame({
    "Nombre d'entreprises": ca_distribution,
    "Pourcentage (%)": (ca_distribution / ca_distribution.sum() * 100).round(2)
})

print(distribution_table)


In [ ]:
fig = px.pie(
    distribution_table.reset_index(),
    names="CA_tranche_affinee",
    values="Nombre d'entreprises",
    title="Répartition des entreprises par tranche de chiffre d'affaires",

)

fig.update_layout(
    width=900,
    height=700,
    legend=dict(
        orientation="v",
        yanchor="top",
        y=1,
        xanchor="left",
        x=1.05
    )
)

fig.show()

In [ ]:
fig = px.bar(
    distribution_table.reset_index(),
    x="CA_tranche_affinee",
    y="Nombre d'entreprises",
    title="Nombre d'entreprises par tranche de chiffre d'affaires",
    labels={
        "index": "Tranche de CA",
        "Nombre d'entreprises": "Nombre d'entreprises"
    }
)

fig.update_layout(
    width=900,
    height=600,
    xaxis_tickangle=-30
)

fig.show()

---